In [1538]:
using LinearAlgebra

In [1539]:
# Example usage:
A = randn(7, 5)

7×5 Matrix{Float64}:
  0.753147   0.565362     0.590011   -0.522405   -0.972895
  0.136851   0.351542    -0.961913   -0.652498    0.331
 -1.21606   -0.833067    -0.905584    0.82773    -0.560869
  0.179738   2.16464      0.959215    1.24051     0.772054
 -0.533359  -0.838793    -0.404893    2.38363     0.836403
 -0.335915   1.06327      0.906434   -0.595917   -0.923025
 -0.805714  -0.00801435   0.0616444  -0.0463836  -1.2036

In [1540]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Returns (inplace) 
    U: input matrix  
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    #U[cidx:end, cidx:end] = U[cidx:end, cidx:end] - 2 * v * (v' * U[cidx:end, cidx:end])
    #show(stdout, "text/plain", I - 2 * v * v')
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Returns (inplace) 
    H: input matrix  
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    
    #V[ridx:end, ridx+1:end] = V[ridx:end, ridx+1:end] - (V[ridx:end, ridx+1:n] * v) * (2 * v')
    #show(stdout, "text/plain", I - 2 * v * v')

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (B = Q_r * A * Q_l)
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
                #print("\ni = ",k,"\n")
                #show(stdout, "text/plain", B)
                #print("\n\n")
                #show(stdout, "text/plain", U * A * V)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U, B, V'
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [1541]:
B, H = bidiagonalize(A);

In [1542]:
show(stdout, "text/plain", B)

7×5 Matrix{Float64}:
 -1.77301       1.9712       1.11022e-16   0.0           1.11022e-16
  2.77556e-17   2.29733      1.78797       0.0           0.0
 -2.22045e-16  -5.55112e-17  1.58271      -1.01315       0.0
  5.55112e-17   0.0          0.0           2.20764      -1.71391
 -1.11022e-16   0.0          0.0           2.22045e-16   1.37324
 -5.55112e-17   2.22045e-16  0.0          -1.38778e-17   0.0
 -1.11022e-16   0.0          0.0          -1.38778e-17   0.0

In [1543]:
show(stdout, "text/plain", H)

7×5 Matrix{Float64}:
  0.844033   -0.884433   -0.234254      0.372063    -0.156435
  0.0457244  -0.734213   -0.707527     -0.136104     0.693456
 -0.406308    0.0857534  -0.873305      0.982635    -0.185548
  0.0600535  -0.246289    0.0997693    -0.961944    -1.0
 -0.178205    0.423183    0.000319838  -0.272541    -0.842233
 -0.112235   -0.432646   -0.234589      0.0170003    0.506119
 -0.269203   -0.163261   -0.415154      0.00977749   0.185706

In [1544]:
_, S1, _ = svd(B);

In [1545]:
S1

5-element Vector{Float64}:
 3.594459797904591
 3.0495938550862745
 2.1247434192987416
 1.2410508568569403
 0.676137889769167

In [1546]:
_, S2, _ = svd(A);

In [1547]:
S2

5-element Vector{Float64}:
 3.5944597979045896
 3.049593855086275
 2.124743419298743
 1.2410508568569398
 0.6761378897691668

In [1548]:
P_l, B, P_r = bidiagonalize(A, true);

In [1549]:
# Checking correctness

In [1550]:
show(stdout, "text/plain", opnorm(B - P_l * A * P_r', 1)/opnorm(A, 1))

5.728463352749384e-16

In [1551]:
show(stdout, "text/plain", opnorm(P_r' * P_r -  I, 1))

4.750301098720859e-16

In [1552]:
show(stdout, "text/plain", opnorm(P_l' * P_l - I, 1))

1.250796927910602e-15

In [1553]:
U, S, V = svd(B)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
7×5 Matrix{Float64}:
 -0.583482      0.201923     -0.674941     -0.333017     -0.228757
 -0.759033      0.101256      0.374791      0.403649      0.331986
 -0.258699     -0.293594      0.555614     -0.456203     -0.5745
  0.125597      0.885321      0.229969      0.104557     -0.369613
 -0.0267895    -0.281039     -0.205897      0.71212      -0.608926
 -5.47586e-17   1.38337e-17  -3.76331e-17   1.15257e-17   1.09272e-16
 -9.46893e-18   9.13488e-19  -2.92591e-17  -4.8805e-17   -9.13965e-17
singular values:
5-element Vector{Float64}:
 3.594459797904591
 3.0495938550862745
 2.1247434192987416
 1.2410508568569403
 0.676137889769167
Vt factor:
5×5 Matrix{Float64}:
  0.287809  -0.805102  -0.49147       0.150057   -0.0701218
 -0.117396   0.206797  -0.0930061     0.738434   -0.624113
  0.56321   -0.220933   0.729258     -0.0259934  -0.318575
  0.475759   0.21826   -0.000260645   0.558417    0.643573
  0.59986    0.461083  -0.466

In [1554]:
P_l' * U

7×5 Matrix{Float64}:
  0.374733    0.0283601  -0.151169   0.384639   0.812218
  0.0523573   0.114034    0.340185  -0.745482   0.386066
 -0.421971    0.228283   -0.468219  -0.36177    0.187661
  0.165136   -0.879591   -0.142644  -0.290408   0.089169
 -0.696761   -0.343005   -0.123147   0.231219   0.18615
  0.405383   -0.0547931  -0.483124  -0.107244  -0.329013
  0.0411434   0.199358   -0.61096   -0.127008   0.0720116

In [1555]:
P_r' * V  

5×5 Matrix{Float64}:
  0.287809  -0.117396   0.56321    0.475759   0.59986
  0.543724  -0.593586  -0.136528  -0.546967   0.184952
  0.379297  -0.341652  -0.261186   0.640119  -0.511309
 -0.633936  -0.585519  -0.322301   0.198805   0.334502
 -0.2753    -0.41751    0.701469  -0.158757  -0.48232

In [1556]:
svd(A)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
7×5 Matrix{Float64}:
 -0.374733   -0.0283601   0.151169  -0.384639  -0.812218
 -0.0523573  -0.114034   -0.340185   0.745482  -0.386066
  0.421971   -0.228283    0.468219   0.36177   -0.187661
 -0.165136    0.879591    0.142644   0.290408  -0.089169
  0.696761    0.343005    0.123147  -0.231219  -0.18615
 -0.405383    0.0547931   0.483124   0.107244   0.329013
 -0.0411434  -0.199358    0.61096    0.127008  -0.0720116
singular values:
5-element Vector{Float64}:
 3.5944597979045896
 3.049593855086275
 2.124743419298743
 1.2410508568569398
 0.6761378897691668
Vt factor:
5×5 Matrix{Float64}:
 -0.287809  -0.543724  -0.379297   0.633936   0.2753
  0.117396   0.593586   0.341652   0.585519   0.41751
 -0.56321    0.136528   0.261186   0.322301  -0.701469
 -0.475759   0.546967  -0.640119  -0.198805   0.158757
 -0.59986   -0.184952   0.511309  -0.334502   0.48232

In [1557]:
U' * P_l * A * P_r' * V   

5×5 Matrix{Float64}:
  3.59446       1.77351e-16   4.61288e-17   7.36378e-16  -3.86466e-16
  2.47131e-16   3.04959       3.36971e-16   1.12335e-16  -7.3393e-16
  2.06903e-16  -7.03992e-16   2.12474       4.41162e-16   5.11039e-16
 -1.075e-15    -5.28849e-16  -2.7659e-16    1.24105       2.86604e-16
 -5.06795e-16   2.70131e-18  -2.99855e-16  -2.24255e-17   0.676138

In [1558]:
show(stdout, "text/plain", B)

7×5 Matrix{Float64}:
 -1.77301       1.9712       1.11022e-16   0.0           1.11022e-16
  2.77556e-17   2.29733      1.78797       0.0           0.0
 -2.22045e-16  -5.55112e-17  1.58271      -1.01315       0.0
  5.55112e-17   0.0          0.0           2.20764      -1.71391
 -1.11022e-16   0.0          0.0           2.22045e-16   1.37324
 -5.55112e-17   2.22045e-16  0.0          -1.38778e-17   0.0
 -1.11022e-16   0.0          0.0          -1.38778e-17   0.0

In [1559]:
function svd_golub_reinsh!(B, maxiter = 80, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing right Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :] 
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    U = zeros(m, m) + I
    V = zeros(n, n) + I
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:maxiter
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            #print(abs(B[j-1, j]), " ", abs(B[j-1, j]) < eps, "\n")
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end
        #print("After scan k = ", k, "\n")
        
        
        #show(stdout, "text/plain", B)
        #print("\n---------------\n")
        
        if k == 1
           # Diagonal matrix, convergence obtained
           return 
        end
        
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check if where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = B[j-1, j] == 0 ? j-1 : l 
            end
            
            l = (l == k) ? 1 : l
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
    
end

svd_golub_reinsh! (generic function with 3 methods)

In [1560]:
B

7×5 Matrix{Float64}:
 -1.77301       1.9712       1.11022e-16   0.0           1.11022e-16
  2.77556e-17   2.29733      1.78797       0.0           0.0
 -2.22045e-16  -5.55112e-17  1.58271      -1.01315       0.0
  5.55112e-17   0.0          0.0           2.20764      -1.71391
 -1.11022e-16   0.0          0.0           2.22045e-16   1.37324
 -5.55112e-17   2.22045e-16  0.0          -1.38778e-17   0.0
 -1.11022e-16   0.0          0.0          -1.38778e-17   0.0

In [1561]:
svd_golub_reinsh!(B, 30)

In [1562]:
B

7×5 Matrix{Float64}:
 -3.59446      -4.24742e-17  -1.02123e-19   9.72358e-19   4.11563e-17
  7.37682e-22   3.04959       8.04182e-18  -1.57628e-21   5.69212e-17
 -9.43853e-17   1.41274e-18   2.12474       9.40894e-19   3.52617e-17
  1.11126e-16   3.08175e-17   2.04154e-18   0.676138     -2.35405e-16
  1.22502e-16   8.22969e-17   1.35178e-16   1.54416e-17   1.24105
 -1.96828e-16   4.21872e-17   7.99606e-17   7.38832e-17   1.4304e-17
 -3.40357e-17   2.78577e-18   6.21681e-17  -6.17966e-17  -6.05695e-17

In [802]:
function givens_rotation(v)
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end

givens_rotation (generic function with 1 method)

In [803]:
A = [1.0 3 4;4 -1 8;7 8 -1]

3×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0

In [804]:
G = givens_rotation(A[[1,2], 1])

2×2 Matrix{Float64}:
 -0.242536  -0.970143
  0.970143  -0.242536

In [805]:
A[[1,2], :] = G * A[[1,2], :];
A

3×3 Matrix{Float64}:
 -4.12311  0.242536  -8.73128
  0.0      3.15296    1.94029
  7.0      8.0       -1.0

In [806]:
G = givens_rotation(A[[1,3], 1])

2×2 Matrix{Float64}:
 0.507519  -0.86164
 0.86164    0.507519

In [807]:
A[[1,3], :] = G * A[[1,3], :];
A


3×3 Matrix{Float64}:
 -8.12404      -6.77003  -3.56965
  0.0           3.15296   1.94029
 -3.33067e-16   4.26913  -8.03075

In [808]:
G = givens_rotation(A[[2,3], 2])

2×2 Matrix{Float64}:
 -0.594089  -0.8044
  0.8044    -0.594089

In [809]:
A[[2,3], :] = G * A[[2,3], :];
A


3×3 Matrix{Float64}:
 -8.12404      -6.77003      -3.56965
  2.67919e-16  -5.30723       5.30723
  1.97871e-16   2.10348e-16   6.33174

In [810]:
qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 8.12404  6.77003   3.56965
 0.0      5.30723  -5.30723
 0.0      0.0       6.33174